In [8]:
#Procesamiento en tiempo real de EPOC+

In [1]:
%cd C:\Users\mgrinberg\Desktop\Matías G\EEG\Captain\Emotiv_BTLE

[Error 3] El sistema no puede encontrar la ruta especificada: u'C:\\Users\\mgrinberg\\Desktop\\Mat\xedas G\\EEG\\Captain\\Emotiv_BTLE'
C:\Users\MATIAS\Desktop\EEG\Mindjam


In [2]:
#
#  Designed for Bluetooth LE on Windows 8/10
#
#  Author:  CaptainSmiley
#
#  Resource:  Warren of project CyKIT
#

try:
    import sys
    import binascii
    import os
    import struct
    from time import *
    from ctypes import *
    from base64 import b64decode
    from ctypes.wintypes import HANDLE, ULONG, DWORD, USHORT
    from Crypto.Cipher import AES
    from Crypto.Util.Padding import pad
except (ImportError, IOError) as e:
    print("ERROR:  %s" % e)
    sys.exit(-1)


DEVICE_UUID = "{81072f40-9f3d-11e3-a9dc-0002a5d5c51b}"
DATA_UUID = "{81072f41-9f3d-11e3-a9dc-0002a5d5c51b}"
MEMS_UUID = "{81072f42-9f3d-11e3-a9dc-0002a5d5c51b}"

BLOCK_SIZE = 16

cipher = ""
start_recording = False

In [3]:
class BTH_LE_GATT_CHARACTERISTIC_VALUE(Structure):
    _fields_ = [
        ("DataSize", c_ulong),
        ("Data", c_ubyte * 20)]         #  this is a hack - 20-bytes should be dynamic, not preallocated

#  -------------------------------------------------------------------
#
#  FUNCTION:  DataCallback (fill_queue Mati G)
#  PURPOSE:   This function is called whenever the bluetooth device sends new
#             information from the headset
#

_CB_FUNC_ = CFUNCTYPE(None, BTH_LE_GATT_CHARACTERISTIC_VALUE)

In [4]:
            
#
#  FUNCTION:  PrintLibError
#  PURPOSE:   This function is called whenever the bluetooth library has an
#             error.  This function replaces the need for Windows-type
#             MessageBox messages
#

_ERR_FUNC_ = CFUNCTYPE(None, c_wchar_p)

def PrintLibError(err):
    print(str(err));

In [5]:
            
#
#  FUNCTION:  PrintLibError
#  PURPOSE:   This function is called whenever the bluetooth library has an
#             error.  This function replaces the need for Windows-type
#             MessageBox messages
#

_ERR_FUNC_ = CFUNCTYPE(None, c_wchar_p)

def PrintLibError(err):
    print(str(err));


def RunApp(EEGqueue):
    global cipher
    
    eegDll = cdll.LoadLibrary(os.getcwd() + "\\EEGBtleLib\\Win32\\Release\\Win32EEGBtleLib.dll")

    #  these are the UUIDs of the data streams we want to decrypt
    #  uuid_list = [unicode(DATA_UUID), unicode(MEMS_UUID)]
    uuid_list = [unicode(DATA_UUID)]
    uuid_clist = (c_wchar_p * len(uuid_list))()
    uuid_clist[:] = uuid_list

    #  set the error callback function to handle lib file errors
    err_func = _ERR_FUNC_(PrintLibError)
    eegDll.set_error_func(err_func)

    #  initialize the dll engine; pass it the UUID of the device we want to connect too
    hDev = eegDll.btle_init(unicode(DEVICE_UUID))

    #  set the callback function to handle the updates
    cb_func = _CB_FUNC_(DataCallback_fill_queue)
    eegDll.set_callback_func(cb_func)

    #  we get the BTLE device ID from the connected device
    name = eegDll.get_bluetooth_id()

    print("Getting connection info...")
    while_timeout = time() + 10
    while c_wchar_p(name).value is None and time() < while_timeout:
        name = eegDll.get_bluetooth_id()
        sleep(1)

    name = c_wchar_p(name).value
    name_id = name[name.find("(")+1:name.find(")")]
    print("Connected headset:  %s" % name)

    #  generate the AES decryption key from the device ID
    sn = bytearray(name_id.decode("hex"))
    k = [sn[-4],sn[-3],sn[-3],sn[-2],sn[-2],sn[-2],sn[-3],sn[-1],sn[-4],sn[-1],sn[-3],sn[-3],sn[-1],sn[-1],sn[-3],sn[-4]]
    AES_key = bytes(bytearray(k)) 

    #print("AES key: " + binascii.hexlify(AES_key) + "\n")

    #  initalize the decryption key function
    cipher = AES.new(AES_key, AES.MODE_ECB)
    
    
    #  start the data collection; pass it the list of UUID characteristics we are
    #  interested in
    eegDll.run_data_collection(hDev, uuid_clist)
    
    for x in range(0, 5):
        sleep(0.1)

    #  disconnect from the device
    eegDll.btle_disconnect(hDev)
    print("Done")

    return 0

In [6]:
def DataCallback_fill_queue(EventOutParameter):
    global start_recording, EEGqueue
    data = bytearray(EventOutParameter.Data)

    #  we want to make sure that our decryption starts with the first of the two btle packets
    if start_recording is False:
        if data[1] == 0x01:
            start_recording = True

    #  decrypt the data stream
    if start_recording is True:
        #print("Ciphertext:\n" + binascii.hexlify(data))
        ciphertext = data[2:18]
        #print("Adjusted:\n" + binascii.hexlify(ciphertext))
        data_out = cipher.decrypt(ciphertext)
        #print("decrypted:\n" + binascii.hexlify(data_out) + "\n")
        data = []  
        for i in range(2, 16, 2):        
            value_1 = int(binascii.hexlify(data_out[i]), 16)
            value_2 = int(binascii.hexlify(data_out[i+1]), 16)
            edk_value = "%.8f" % (((value_1 * .128205128205129) + 4201.02564096001) + ((value_2 - 128) * 32.82051289))

            if data[1] == 0x01:
                data.append((count, edk_value))
                #EEG_queue.put((count, edk_value))
                #print("[%d]: %s" % (count, edk_value))
            if data[1] == 0x02:
                data.append((count+8, edk_value))
                #EEG_queue.put((count+8, edk_value))
                #print("[%d]: %s" % (count+8, edk_value))
        
        try:
            EEG_queue.put(data, block=True, timeout = None)
            EEG_queue.task_done()
            
        except Full:
            print "Filler exiting, timeout/full"
            sleep(0.1)
        if n >= limit:
            print "Filler exiting, reached limit - %s" % limit
            sleep(0.1)

In [2]:
from multiprocessing import Process, Queue
from time import sleep

In [3]:
class RealtimeEEG(object):
    """Recieves realtime data, preprocessing and does something"""
    
    def __init__(self, EEGqueue):
        # self.sampling_rate = 128  #Hz
        # self.number_of_channels = 14
        # self.eeg_chunk_size_seconds = 5 #s
        # self.eeg_timesteps = sampling_rate*realtime_eeg_in_second
        # self.socket_port = 8080
        #self.channel_names = "F3 FC5 AF3 F7 T7 P7 O1 O2 P8 T8 F8 AF4 FC6 F4"
        self.data_queue = data_queue
        
    def process_data(self):
        pass
    def put_data(self):
        
    
    def run(self):
        processes = [Process(target=self.process_data, args=(self.data_queue,)), Process(target=self.put_data, args=(EEG_queue,))]

        for t in threads:
            t.start()

        for t in threads:
            t.join()
            
        def do_fft(self, data):
            fs = 512                                
            data = np.random.uniform(0, 100, 1024)  # 2 sec of data b/w 0.0-100.0

            # Get real amplitudes of FFT (only in postive frequencies)
            fft_vals = np.absolute(np.fft.rfft(data))

            # Get frequencies for amplitudes in Hz
            fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)

            eeg_bands = {'Delta': (0, 4),
                         'Theta': (4, 8),
                         'Alpha': (8, 12),
                         'Beta': (12, 30),
                         'Gamma': (30, 45)}

            # Take the mean of the fft amplitude for each EEG band
            eeg_band_fft = dict()
            for band in eeg_bands:  
                freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                                   (fft_freq <= eeg_bands[band][1]))[0]
                eeg_band_fft[band] = np.mean(fft_vals[freq_ix])

        
    def plot_bands(self):
        # Plot the data (using pandas here cause it's easy)
        import pandas as pd
        df = pd.DataFrame(columns=['band', 'val'])
        df['band'] = eeg_bands.keys()
        df['val'] = [eeg_band_fft[band] for band in eeg_bands]
        ax = df.plot.bar(x='band', y='val', legend=False)
        ax.set_xlabel("EEG band")
        ax.set_ylabel("Mean band Amplitude")
    
    def mne_plot(self):
        pass
    def compute_features(self):
        pass
    def predict():
        pass

In [1]:
data_queue = Queue()

NameError: name 'Queue' is not defined

In [19]:
EEG = RealtimeEEG(EEG_queue)

In [20]:
EEG.run()

NameError: global name 'threads' is not defined